## Workflow jobs
- Mục đích:
    - xác định các đơn vị công việc thực hiện nhiệm vụ cụ thể trong workflow
- Yếu tố:
    - jobs: Keyword cho công việc cụ thể
    - job_name: Tên công việc
    - runs-on: Hệ điều hành của máy chủ chạy công việc (ubuntu-latest, windows-latest, macos-latest)
    - steps: Danh sách các bước thực hiện công việc

Sample:
- Workflow
    - Job
        - Step
            - Action

VD:
```yaml
name: Example workflow
on:
    push:
        branches:
            - main
job:
    example-job: # Job name
        runs-on: ubuntu-latest # Target OS
        steps: # List of steps
            - name: Check code
              run: actions/checkout@v2

            - name: Execute a command
              run: echo "Hello, world!"
```

## Sử dụng job steps cho actions và shell commands

- Mục đích:
    - Xác định nhiệm vụ riêng lẻ sẽ được thực hiện trong job

- Yếu tố:
    - steps: Danh sách các bước thực hiện công việc
    - uses: (Optional) Sử dụng pre-built action từ Github Actions Marketplace
    - action_name: Tên action
    - version: (Optional) Phiên bản của action
    - with: (Optional) Thông tin cấu hình cho action
    - name: (Optional) Tên bước thực hiện
    - run: Command shell sẽ thực thi

VD:
```yaml
name: Example Workflow for Job Steps
on:
    push:
        branches:
            - main
jobs:
    example-job:
        runs-on: ubuntu-latest
        steps:
            - name: Check Repository # name
              uses: actions/checkout@v2 # uses

            - name: Set up Node.js # name
              uses: actions/setup-node@v2 # action_name
              with: # with: Giá trị đưa vào cho action
                node-version: '14'

            - name: Install Dependence # name
              run: npm install # run

            - name: Run Test # name
              run: npm test # command_to_execute: Shell command
```

## Sử dụng conditional keywords cho steps

- Mục đích: Kiểm soát các bước thực thi trong job dựa trên điều kiện cụ thể
- Keywords:
    - if: Điều kiện thực thi nếu đúng
    - else: Điều kiện thực thi nếu sai
    - needs: Danh sách các công việc cần chạy trước khi công việc hiện tại chạy

VD:
```yaml
jobs:
    initial_job:
        runs-on: ubuntu-latest
        steps:
        - name: Check out code
          uses: actions/checkout@v2
        - name: Run a command
          run: echo "Hello, world!"
    conditional_job:
        runs-on: ubuntu-latest
        needs: initial_job
        steps:
        - name: Execute on specific condition
          run: echo "This step runs because the condition is true."
          if: github.ref == 'refs/heads/main' # Conditional keyword

        - name: Alternative step for false condition
          run: echo "This step would run if the above condition is false."
          if: github.ref != 'refs/heads/main' # Conditional keyword
```

## Mô tả cách actions, workflows, job, step, runs và marketplace hoạt động với nhau

- Workflow: Một tập hợp các công việc và các bước thực hiện công việc
- Job: Một đơn vị công việc cụ thể trong workflow, mỗi job có một step riêng
- Step: Nhiệm vụ đơn lẻ trong một job, được kích hoạt bởi một action hoặc command shell
- Actions: Pre-built script cung cấp các công cụ và chức năng cụ thể, có thể được sử dụng trong các bước thực hiện công việc (Marketplace)
- Shell commands: Script shell thực thi các lệnh cụ thể
- Runs: Một kích hoạt của một workflow, bao gồm các công việc và bước thực hiện công việc trigger bởi sự kiện cụ thể
- Marketplace: Nơi lưu trữ các actions sẵn có để sử dụng trong workflows

## Xác định kịch bản phù hợp khi sử dụng Github-hosted và self-hosted runners

- Github-hosted runners: Máy chủ được quản lý bởi Github, cung cấp môi trường chạy công việc, không cần phải quản lý máy chủ riêng
    - Free to use
    - Giới hạn tài nguyên
    - Pre-configured với nhiều ứng dụng
    - Phù hợp với workfloww đơn giản và các dự án open-source
- Self-hosted runners: Máy chủ do người dùng tự quản lý, cung cấp môi trường chạy công việc, cần phải quản lý máy chủ riêng
    - Nhiều quyền điều khiển và tùy chỉnh
    - Có thể sử dụng để chạy workflow cho một cấu hình hoặc ứng dụng cụ thể
    - Cần phải quản lý và duy trì máy chủ

## Triển khai workflow dưới dạng runs step

- Mục đích:
    - Tương tác với runner trong khi thực thi workflow
- Commands:
    - set-output: Lưu trữ giá trị đầu ra từ một step để sử dụng trong các bước khác
    - upload-artifact: Tải lên một tệp hoặc thư mục nhất định để sử dụng trong các bước khác hoặc trong các công việc khác
    - download-artifact: Tải xuống một tệp hoặc thư mục đã được tải lên trước đó
    - echo: In ra màn hình console

VD:
```yaml
steps:
    - name: Set an output variable
      run: echo "My output is: $MY_VARIABLE"
    - name: Upload artifact
      uses: actions/upload-artifact@v2
      with:
        name: my-artifact
        path: path/to/artifact
    - name: Download artifact
      uses: actions/download-artifact@v2
      with:
        name: my-artifact
        path: path/to/download
```

## Mô tả việc sử dụng các dependent jobs trong workflow

- Mục đích:
    - Đảm bảo rằng các nhiệm vụ cụ thể được thực hiện bởi các yêu cầu tương ứng
- Lợi ích:
    - Cải thiện workflow control và execution flow
    - Phòng tránh lỗi hoặc sự không nhất quán bằng cách đảm bảo rằng các công việc cần thiết được thực hiện trước khi công việc khác bắt đầu

VD:
```yaml
jobs:
    build:
        runs-on: ubuntu-latest
        steps:
        - name: Check out code
          uses: actions/checkout@v2
        - name: Build the project
          run: make build
          with:
            name: build
            path: ./build

    deploy:
        needs: build # Dependent job
        runs-on: ubuntu-latest
        steps:
            - uses: actions/checkout@v2
              with:
                name: build
                path: ./build
            - run: make deploy
```